In [ ]:
import pandas as pd
import altair as alt
from pathlib import Path

In [ ]:
rna_count_folder = '../Data/RNAscoring_dev_data/RNA_counts/' #path to RNA counts folder
dna_count_folder = '../Data/RNAscoring_dev_data/DNA_counts/' #paths to DNA counts folder

In [ ]:
def read_counts(rna_counts, dna_counts):
    rna_path = Path(rna_counts) #Creates path object to RNA counts
    dna_path = Path(dna_counts) #Creates path object to DNA counts
    
    rna_outputs = list(rna_path.glob('*.tsv')) #Gets list of RNA count files
    dna_outputs = list(dna_path.glob('*.tsv')) #Gets list of DNA count files
    
    all_outputs = []

    i = 0
    while i < len(rna_outputs):
        paired_output = (rna_outputs[0], dna_outputs[0])
        all_outputs.append(paired_output)

        i += 1

    for elem in all_outputs:
        rna, dna = elem

        rna_counts = pd.read_csv(rna, sep = '\t')
        sampleid = rna_counts['sampleid'][0]
        
        if 'R3R6' or 'R7R8R9' in sampleid:
            rep = 'rep_3'
        elif 'R2R5' or 'R4R5R6' in sampleid:
            rep = 'rep_2'
        elif 'R1R4' or 'R1R2R3' in sampleid:
            rep = 'rep_1'
            
        target_rep_name = rna_counts['target'] + rep
        rna_counts['target_rep'] = target_rep_name
        rna_counts = rna_counts.rename(columns = {'count': 'RNAcount'})
        rna_counts['pos'] = rna_counts['pos'].astype(str)
        rna_counts['posID'] = rna_counts['pos'] + ':' + rna_counts['allele']

        
        dna_counts = pd.read_csv(dna, sep = '\t')
        dna_counts = dna_counts.rename(columns = {'count': 'DNAcount'})
        dna_counts['pos'] = dna_counts['pos'].astype(str)
        dna_counts['posID'] = dna_counts['pos'] + ':' + dna_counts['allele']
        
        merged = pd.merge(rna_counts, dna_counts, how = 'inner', on = 'posID')
        merged = merged.drop(columns = ['sampleid_x', 'sampleid_y', 'target_x', 'target_y','chrom_x', 'chrom_y', 'pos_x', 'pos_y', 'allele_x', 'allele_y'])
        
        print(merged)

In [ ]:
def main():
    read_counts(rna_count_folder, dna_count_folder)

In [ ]:
main()